<a href="https://colab.research.google.com/github/b-fatma/S2I-DM/blob/master/src/merge/merge_add_elevation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import geopandas as gpd
import pandas as pd
import csv
import sys
import geopandas as gpd
import shapely

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Loading the Data

In [14]:
fire_landcover_path = '/content/drive/MyDrive/dm_fire_prediction/merge/fire_landcover.csv'
elevation_path = '/content/drive/MyDrive/dm_fire_prediction/premerge/elevation.csv'

In [15]:
fl_df = pd.read_csv(fire_landcover_path)
el_df = pd.read_csv(elevation_path)

fl_df.head(), el_df.head()

(   longitude  latitude  fire                  geometry  lcccode_label
 0    5.53337  35.70751     1  POINT (5.53337 35.70751)           16.0
 1    6.46961  32.13579     1  POINT (6.46961 32.13579)           19.0
 2    6.97630  32.35563     1   POINT (6.9763 32.35563)           19.0
 3    9.39581  28.19791     1  POINT (9.39581 28.19791)           18.0
 4    9.49323  28.12826     1  POINT (9.49323 28.12826)           18.0,
    longitude   latitude  elevation_ED_smoothed_by_mean
 0   8.956111  37.556111                         -280.0
 1   8.947778  37.551944                         -280.0
 2   8.943611  37.539444                         -280.0
 3   8.947778  37.539444                         -280.0
 4   8.939444  37.535278                         -280.0)

In [16]:
fl_gdf = gpd.GeoDataFrame(fl_df, geometry=gpd.points_from_xy(fl_df.longitude, fl_df.latitude))
el_gdf = gpd.GeoDataFrame(el_df, geometry=gpd.points_from_xy(el_df.longitude, el_df.latitude))

fl_gdf.head(), el_gdf.head()

(   longitude  latitude  fire                  geometry  lcccode_label
 0    5.53337  35.70751     1  POINT (5.53337 35.70751)           16.0
 1    6.46961  32.13579     1  POINT (6.46961 32.13579)           19.0
 2    6.97630  32.35563     1   POINT (6.9763 32.35563)           19.0
 3    9.39581  28.19791     1  POINT (9.39581 28.19791)           18.0
 4    9.49323  28.12826     1  POINT (9.49323 28.12826)           18.0,
    longitude   latitude  elevation_ED_smoothed_by_mean  \
 0   8.956111  37.556111                         -280.0   
 1   8.947778  37.551944                         -280.0   
 2   8.943611  37.539444                         -280.0   
 3   8.947778  37.539444                         -280.0   
 4   8.939444  37.535278                         -280.0   
 
                    geometry  
 0  POINT (8.95611 37.55611)  
 1  POINT (8.94778 37.55194)  
 2  POINT (8.94361 37.53944)  
 3  POINT (8.94778 37.53944)  
 4  POINT (8.93944 37.53528)  )

## Merge

In [18]:
# Perform nearest-neighbor join
joined = fl_gdf.sjoin_nearest(
    el_gdf[['elevation_ED_smoothed_by_mean', 'geometry']],
    how='left'
)

# If fl_gdf index structure must be preserved:
fl_gdf['elevation_ED_smoothed_by_mean'] = joined['elevation_ED_smoothed_by_mean']

fl_gdf.head(), joined.head()


(   longitude  latitude  fire                  geometry  lcccode_label  \
 0    5.53337  35.70751     1  POINT (5.53337 35.70751)           16.0   
 1    6.46961  32.13579     1  POINT (6.46961 32.13579)           19.0   
 2    6.97630  32.35563     1   POINT (6.9763 32.35563)           19.0   
 3    9.39581  28.19791     1  POINT (9.39581 28.19791)           18.0   
 4    9.49323  28.12826     1  POINT (9.49323 28.12826)           18.0   
 
    elevation_ED_smoothed_by_mean  
 0                         1787.5  
 1                         -280.0  
 2                         -280.0  
 3                          581.0  
 4                          581.0  ,
    longitude  latitude  fire                  geometry  lcccode_label  \
 0    5.53337  35.70751     1  POINT (5.53337 35.70751)           16.0   
 1    6.46961  32.13579     1  POINT (6.46961 32.13579)           19.0   
 2    6.97630  32.35563     1   POINT (6.9763 32.35563)           19.0   
 3    9.39581  28.19791     1  POINT (9.3

In [20]:
fl_gdf['elevation_ED_smoothed_by_mean'].value_counts()

,count
elevation_ED_smoothed_by_mean,
-280.0,572140
388.0,486858
581.0,486183
1787.5,420076


## Saving the Output

In [21]:
output_path = '/content/drive/MyDrive/dm_fire_prediction/merge/fire_landcover_elevation.csv'

fl_gdf.drop(columns=['geometry']).to_csv(output_path, index=False)